In [1]:
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# create list of multiple dataframes containing participants' data
path =r'C:\Users\lttso\Desktop\test_data'
filenames = glob.glob(path + "/*.csv")
dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename))

In [3]:
#shows an example of pure df to work with
dfs[0]

,block_n,trial_n,training_trials,target1,target2,back_to_center,num_of_targets,condition,stim0,x_0,...,y_targ,targ_selected,valid_click,participant,age,gender,date,expName,psychopyVersion,Unnamed: 42
0,0,0,1,red_cc,green_cc,False,4,feat,green_cc,88.798470,...,-78.508362,True,True,Testsub1,23,f,2022_Aug_10_1244,foraging_circle,2021.2.3,NaN
1,0,0,1,red_cc,green_cc,False,3,feat,green_cc,46.377595,...,-151.933354,True,True,Testsub1,23,f,2022_Aug_10_1244,foraging_circle,2021.2.3,NaN
2,0,0,1,red_cc,green_cc,False,4,feat,yellow_cc,93.587744,...,-73.807224,True,True,Testsub1,23,f,2022_Aug_10_1244,foraging_circle,2021.2.3,NaN
3,0,0,1,red_cc,green_cc,False,3,feat,yellow_cc,137.414418,...,-80.483495,True,True,Testsub1,23,f,2022_Aug_10_1244,foraging_circle,2021.2.3,NaN
4,0,0,1,red_cc,green_cc,False,2,feat,green_cc,49.157680,...,-1.557801,True,True,Testsub1,23,f,2022_Aug_10_1244,foraging_circle,2021.2.3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2070,3,31,0,yellow_cc,blue_cc,False,4,feat,blue_cc,495.944003,...,-465.911057,True,True,Testsub1,23,f,2022_Aug_10_1244,foraging_circle,2021.2.3,NaN
2071,3,31,0,yellow_cc,blue_cc,True,3,feat,green_cc,87.676453,...,NaN,NaN,False,Testsub1,23,f,2022_Aug_10_1244,foraging_circle,2021.2.3,NaN
2072,3,31,0,yellow_cc,blue_cc,True,3,feat,green_cc,87.676453,...,-82.203575,True,True,Testsub1,23,f,2022_Aug_10_1244,foraging_circle,2021.2.3,NaN
2073,3,31,0,yellow_cc,blue_cc,False,5,feat,yellow_cc,137.178073,...,-73.044200,True,True,Testsub1,23,f,2022_Aug_10_1244,foraging_circle,2021.2.3,NaN


In [5]:
# Create a list to store cleaned dataframes
dfs_cleaned = []

# Get age and gender for further count
gender = []
age = []

#create dictionary with error rates
count = 1
error_rates_feat = dict()
error_rates_conj = dict()
for df in dfs:
    # record age and gender for further count
    gen = df.gender.unique()
    a = df.age.unique()
    gender.append(gen[0])
    age.append(a[0])
    
    #clean df from unrelated data & columns
    df = df[df.training_trials != 1]
    df = df[df.valid_click != False]
    df = df.drop(['gender', 'age', 'date', 
                  'psychopyVersion', 'valid_click', 'expName'], axis=1) # add "Unnamed: 42" for lab data
    
    #create dfs for feature and conjunction
    df_feat = df[df['condition'] == 'feat']
    df_conj = df[df['condition'] == 'conj']
    
    
    error_selections = df.loc[(df['targ_selected'] == False), 'trial_n'].tolist()
    # calculate error rates for feture 
    actual_num_of_trials = 32 * 4
    error_select_feat = df_feat.loc[(df_feat['targ_selected'] == False), 'trial_n'].tolist()
    error_selections_feat = len(error_select_feat)
    rates_feat = 100 - (abs(error_selections_feat - actual_num_of_trials) / actual_num_of_trials * 100)
    #store error rates in dict('participant_n':error rate)
    error_rates_feat.update({str(count): rates_feat})
    
    # calculate error rates for conj 
    actual_num_of_trials = 32 * 4
    error_select_conj = df_conj.loc[(df_conj['targ_selected'] == False), 'trial_n'].tolist()
    error_selections_conj = len(error_select_conj)
    rates_conj = 100 - (abs(error_selections_conj - actual_num_of_trials) / actual_num_of_trials * 100)
    #store error rates in dict('participant_n':error rate)
    error_rates_conj.update({str(count): rates_conj})
    
    
    count += 1
    df = df[~df['trial_n'].isin(error_selections)]

    
    #create lists of dataframes
    dfs_cleaned.append(df)

In [6]:
error_rates_feat

{'1': 10.9375, '2': 6.25, '3': 7.03125, '4': 0.78125, '5': 9.375}

In [7]:
error_rates_conj

{'1': 17.96875, '2': 5.46875, '3': 1.5625, '4': 0.78125, '5': 20.3125}

In [8]:
dfs_cleaned[0]

,block_n,trial_n,training_trials,target1,target2,back_to_center,num_of_targets,condition,stim0,x_0,...,y_resp,click_time,direction,switch,selected_stim,x_targ,y_targ,targ_selected,participant,Unnamed: 42
227,0,4,0,red_cc,green_sq,False,3,conj,green_cc,91.864636,...,-78.0,0.701358,4.0,0.0,green_sq,-41.138228,-75.499841,True,Testsub1,NaN
229,0,4,0,red_cc,green_sq,False,2,conj,red_sq,47.112806,...,-160.0,0.884061,4.0,0.0,green_sq,-89.678923,-159.349127,True,Testsub1,NaN
230,0,4,0,red_cc,green_sq,False,4,conj,red_cc,2.731052,...,-158.0,0.800981,3.0,0.0,green_sq,-179.818411,-158.954441,True,Testsub1,NaN
231,0,4,0,red_cc,green_sq,False,2,conj,red_sq,-90.547766,...,-227.0,0.467124,5.0,0.0,green_sq,-135.749622,-230.008864,True,Testsub1,NaN
232,0,4,0,red_cc,green_sq,False,4,conj,green_sq,-42.703718,...,-234.0,0.983840,0.0,0.0,green_sq,-42.703718,-236.750001,True,Testsub1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,3,23,0,yellow_cc,blue_cc,False,3,feat,blue_cc,403.338489,...,77.0,0.533009,0.0,0.0,blue_cc,403.338489,77.422496,True,Testsub1,NaN
1995,3,23,0,yellow_cc,blue_cc,False,3,feat,red_cc,493.918591,...,98.0,0.800674,3.0,0.0,blue_cc,319.285074,82.883432,True,Testsub1,NaN
1996,3,23,0,yellow_cc,blue_cc,False,2,feat,yellow_cc,409.839929,...,-13.0,0.450101,4.0,0.0,blue_cc,270.963010,-2.963881,True,Testsub1,NaN
1997,3,23,0,yellow_cc,blue_cc,False,3,feat,green_cc,360.146475,...,-95.0,0.350440,4.0,0.0,blue_cc,225.553223,-79.367775,True,Testsub1,NaN


In [9]:
# function to calculate the number of targets per selection
def targets_per_selection(row):
    num_of_targets = 0
    for i in range(0, 6):
        if row['target1'] == row['stim' + str(i)] or row['target2'] == row['stim' + str(i)]:
            num_of_targets += 1
    return num_of_targets

In [91]:
# New column with the number of targets in selection patch  
df['num_of_targets'] = df.apply(lambda row: targets_per_selection(row), axis=1)
# New column with the ratio of targets to distractors in selection patch
df['ratio'] = df.apply(lambda row: row.num_of_targets / 6, axis = 1)
df.drop(['Unnamed: 42'], axis=1)

,block_n,trial_n,training_trials,target1,target2,back_to_center,num_of_targets,condition,stim0,x_0,...,y_resp,click_time,direction,switch,selected_stim,x_targ,y_targ,targ_selected,participant,ratio
173,0,1,0,red_cc,green_sq,False,4,conj,red_cc,87.968991,...,-86.0,0.685103,4.0,0.0,red_cc,-40.751132,-80.217743,True,Testsub1,0.666667
175,0,1,0,red_cc,green_sq,False,5,conj,red_cc,46.034454,...,-83.0,0.767245,0.0,0.0,red_cc,46.034454,-75.501230,True,Testsub1,0.833333
176,0,1,0,red_cc,green_sq,False,5,conj,green_sq,131.783242,...,-79.0,0.717223,3.0,0.0,red_cc,-45.715039,-80.054481,True,Testsub1,0.833333
177,0,1,0,red_cc,green_sq,False,4,conj,green_sq,43.538125,...,-85.0,1.050829,0.0,1.0,green_sq,43.538125,-82.821065,True,Testsub1,0.666667
178,0,1,0,red_cc,green_sq,False,4,conj,red_cc,138.155957,...,-83.0,0.533867,3.0,0.0,green_sq,-42.612047,-74.901767,True,Testsub1,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1877,3,26,0,red_cc,green_sq,False,6,conj,red_cc,315.887294,...,161.0,0.483807,5.0,0.0,green_sq,272.262017,160.058487,True,test_sub_5,1.000000
1878,3,26,0,red_cc,green_sq,False,3,conj,red_sq,359.709912,...,86.0,0.783211,4.0,1.0,red_cc,221.995087,80.219193,True,test_sub_5,0.500000
1879,3,26,0,red_cc,green_sq,False,5,conj,green_sq,315.755818,...,93.0,1.066487,3.0,0.0,red_cc,137.197355,80.828312,True,test_sub_5,0.833333
1880,3,26,0,red_cc,green_sq,False,3,conj,red_cc,226.208585,...,83.0,0.849351,3.0,1.0,green_sq,48.433759,73.776567,True,test_sub_5,0.500000


In [92]:
#merge dataframes in one big df
df = pd.concat(dfs_cleaned)

In [94]:
df.to_csv (r'C:\Users\lttso\Desktop\foraging\foraging_cirle.csv', index = False,  mode='a', header=True)